<a href="https://colab.research.google.com/github/mehtashish778/The-Uyghur-Voice-Cup/blob/main/notebookbc3e522506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

the_uyghur_voice_cup_path = kagglehub.competition_download('the-uyghur-voice-cup')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch torchaudio librosa transformers datasets huggingface_hub numpy pandas matplotlib seaborn
!pip install evaluate
!pip install jiwer

In [ ]:
from datasets import load_dataset
import librosa
import numpy as np
import pandas as pd
import torch

# Load dataset from local CSV files
dataset = load_dataset("csv", data_files={"train": "/kaggle/input/the-uyghur-voice-cup/train.csv", "test": "/kaggle/input/the-uyghur-voice-cup/test.csv"})

# Train and test splits
train_data = dataset["train"]  # Contains 'transcription'
test_data = dataset["test"]  # Does not contains "transcription"

In [ ]:
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    AutoModelForCTC,
    Wav2Vec2CTCTokenizer,
)
import torch



# Extract all characters from the entire dataset (no need for .map here)
all_text = " ".join(dataset["train"]["transcription"])
vocab = list(set(all_text))

# Create vocab dictionary
vocab_dict = {v: k for k, v in enumerate(sorted(vocab))}
vocab_dict["|"] = vocab_dict.pop(" ")  # Replace space with |
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")



feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
import torch
import numpy as np
import librosa
import os # Import the os module

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the base path to the unzipped data
BASE_DATA_PATH = "/kaggle/input/the-uyghur-voice-cup"

# def prepare_sample(batch):
#     # Construct the full audio file path
#     audio_path = os.path.join(BASE_DATA_PATH, batch["filepath"])  # Join base path with relative filepath
#     audio, sampling_rate = librosa.load(audio_path, sr=16000)     # Load audio file, resample to 16kHz

#     expected_length = 32000  # 2 seconds of audio at 16kHz (not 32!)

#     # Trim or pad to 2 seconds
#     if len(audio) < expected_length:
#         audio = np.pad(audio, (0, expected_length - len(audio)), mode='constant')
#     else:
#         audio = audio[:expected_length]

#     # Process raw audio into model-ready format
#     inputs = processor(audio, sampling_rate=16000, return_tensors="pt")

#     batch["input_values"] = inputs["input_values"].squeeze(0)  # ✅ Only use this
#     batch["transcription"] = batch["transcription"]

#     return batch


def prepare_sample(batch):
    audio_path = os.path.join(BASE_DATA_PATH, batch["filepath"])
    audio, _ = librosa.load(audio_path, sr=16000)

    # Process raw waveform into model input
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt")

    batch["input_values"] = inputs["input_values"].squeeze(0)
    batch["transcription"] = batch["transcription"]

    return batch



In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Union
import torch

In [ ]:
from datasets import load_dataset

# Load your dataset from CSV
dataset = load_dataset("csv", data_files="/kaggle/input/the-uyghur-voice-cup/train.csv")["train"]

# Optional: Take a sample or full dataset
sampled_dataset = dataset  # Or use `.select(...)` or `.shuffle(...)` if needed

# Apply preprocessing
sampled_dataset = sampled_dataset.map(
    prepare_sample,
    remove_columns=["ID", "filepath"],  # Updated to remove only "ID" and "filepath"
    desc="Preprocessing audio samples"
)

# Set the format of the dataset to 'torch' to ensure tensors are handled correctly
sampled_dataset.set_format(type='torch', columns=['input_values', 'transcription'])

In [ ]:
sample = sampled_dataset[0]
print("Transcript:", sample["transcription"])
print("Token IDs:", processor.tokenizer(sample["transcription"]).input_ids)

In [ ]:
# Split dataset
dataset1 = sampled_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)


from typing import List, Dict, Union
import torch
import torch.nn.utils.rnn

# class DataCollatorWithPadding:
#     def __init__(self, processor, padding: Union[bool, str] = True):
#         self.processor = processor
#         self.padding = padding

#     def __call__(self, features: List[Dict[str, Union[torch.Tensor, str]]]) -> Dict[str, torch.Tensor]:
#         # Convert input values to tensors if not already
#         input_values = [f["input_values"].clone().detach() for f in features]


#         # Tokenize transcriptions (labels)
#         labels = self.processor(
#             text=[f["transcription"] for f in features],
#             padding=self.padding,
#             return_tensors="pt"
#         ).input_ids

#         # Replace pad tokens with -100 (ignored by CTC loss)
#         labels[labels == self.processor.tokenizer.pad_token_id] = -100

#         # Pad audio input sequences
#         padded_input_values = torch.nn.utils.rnn.pad_sequence(
#             input_values, batch_first=True,
#             padding_value=self.processor.feature_extractor.padding_value
#         )

#         return {
#             "input_values": padded_input_values,  # Wav2Vec2 expects this key
#             "labels": labels
#         }

class DataCollatorWithPadding:
    def __init__(self, processor, padding=True):
        self.processor = processor
        self.padding = padding

    def __call__(self, features):
        input_values = [f["input_values"].clone().detach() for f in features]

        # Tokenize transcriptions
        transcriptions = [f["transcription"] for f in features]
        label_inputs = self.processor.tokenizer(
            transcriptions,
            padding=True,
            return_tensors="pt",
            return_attention_mask=False,
        )
        labels = label_inputs.input_ids
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        padded_input_values = torch.nn.utils.rnn.pad_sequence(
            input_values,
            batch_first=True,
            padding_value=self.processor.feature_extractor.padding_value
        )

        return {
            "input_values": padded_input_values,
            "labels": labels,
        }


In [ ]:
data_collator = DataCollatorWithPadding(processor, padding=True)


import torch
from evaluate import load

# Load ASR metrics
wer_metric = load("wer")
cer_metric = load("cer")


def compute_metrics(pred):
    pred_ids = np.argmax(pred.predictions, axis=-1)
    label_ids = pred.label_ids

    # Replace -100 with pad_token_id for decoding labels properly
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode to strings
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Compute metrics
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {
        "wer": wer,
        "cer": cer
    }



In [ ]:
!pip install jiwer

In [ ]:
from transformers import TrainingArguments, Trainer


In [ ]:
model.to("cuda")

model.freeze_feature_encoder()
# Or freeze lower transformer layers manually
for name, param in model.named_parameters():
    if "encoder.layer." in name and any(layer in name for layer in ["0", "1", "2", "3","4"]):
        param.requires_grad = False
training_args = TrainingArguments(
  output_dir="AsrTaskModel",
  group_by_length=True,
  per_device_train_batch_size=2,
  per_device_eval_batch_size=2,
  eval_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=1000,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  load_best_model_at_end=True,
  save_strategy="steps",
  report_to="none",
  remove_unused_columns=False,
)

# Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset1["train"],
    eval_dataset=dataset1["test"],
    tokenizer=processor.feature_extractor,
)

trainer.train()

In [ ]:
import torch
import pandas as pd
import numpy as np
from scipy.special import softmax
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure model is in evaluation mode
model.eval()

# Prepare a list to store submission results
submission_results = []

In [ ]:

for sample in tqdm(test_data):
    sample_id = sample["ID"]  # Keep original ID
    audio_input = sample["filepath"]

    # Load audio
    audio_path = os.path.join(BASE_DATA_PATH, audio_input)
    audio, _ = librosa.load(audio_path, sr=16000)

    # Preprocess audio
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    input_values = inputs.input_values.to(model.device)

    # Run inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode prediction (greedy)
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Save (id, transcription)
    submission_results.append([sample_id, transcription])


In [ ]:
# Convert to DataFrame (keeping only "id" and "transcription")
submission_df = pd.DataFrame(submission_results, columns=["ID", "transcription"])

# Save to CSV without modifying id
submission_df.to_csv("./submission.csv", index=False)

print("✅ Submission file saved correctly with original 'id' values!")